# Network Evaluation

## Imports

In [1]:
from tablut.game import Board, draw_board
import random
import keras
import tensorflow as tf
import numpy as np
from tqdm.notebook import tqdm

## Simulate Games

Results:
Random vs Random = 0.85 / 15 / 0

Random Network vs Random = 0.82 / 0.18 / 0.0\
Random vs Random Network: 0.84 / 0.16 / 0.0

Supervised Network 1D vs Random = 0.94 / 0.06 / 0.0\
Random vs Supervised Network 1D = 0.1 / 0.9 / 0.0

Supervised Network MD vs Random: 0.94 / 0.06 / 0.0\
Random vs Supervised Network MD: 0.13 / 0.87 / 0.0

Random Network vs Random Network: 0.33 / 0.19 / 0.48\
Supervised Network 1D vs Supervised Network 1D: 0.34 / 0.49 / 0.17\


In [ ]:
#model = keras.models.load_model('saved_models/random_model_tablut.keras')
#model = keras.models.load_model('saved_models/supervised_model_tablut.keras')
model = keras.models.load_model('saved_models/supervised_model_tablut_md.keras')

In [5]:
def fst(a):
    return a[0]

# white == random player
# black == network
def rand_vs_net(board):
    record = []
    while(not fst(board.is_terminal())):
        if(board.turn == Board.WHITE):
            moves = board.generate_moves()
            m = moves[random.randint(0, len(moves)-1)]
            board.apply_move(m)
            record.append(m)
            continue
        else:
            q = model.predict(np.array([board.to_network_input_multidim()]))
            masked_output = [0 for x in range(0,1296)]
            for m in board.generate_moves():
                m_idx = board.get_network_output_index(m)
                masked_output[m_idx] = q[0][0][m_idx]
            best_idx = np.argmax(masked_output)
            sel_move = None
            for m in board.generate_moves():
                m_idx = board.get_network_output_index(m)
                if(best_idx == m_idx):
                    sel_move = m
            board.apply_move(sel_move)
            record.append(sel_move)
            continue
    terminal, winner = board.is_terminal()
    return winner

# white == network
# black == random player
def net_vs_rand(board):
    record = []
    while(not fst(board.is_terminal())):
        if(board.turn == Board.BLACK):
            moves = board.generate_moves()
            m = moves[random.randint(0, len(moves)-1)]
            board.apply_move(m)
            record.append(m)
            continue
        else:
            q = model.predict(np.array([board.to_network_input_multidim()]))
            masked_output = [0 for x in range(0,1296)]
            for m in board.generate_moves():
                m_idx = board.get_network_output_index(m)
                masked_output[m_idx] = q[0][0][m_idx]
            best_idx = np.argmax(masked_output)
            sel_move = None
            for m in board.generate_moves():
                m_idx = board.get_network_output_index(m)
                if(best_idx == m_idx):
                    sel_move = m
            board.apply_move(sel_move)
            record.append(sel_move)
            continue
    terminal, winner = board.is_terminal()
    return winner

# white random player
# black random player
def rand_vs_rand(board):
    while(not fst(board.is_terminal())):
        moves = board.generate_moves()
        m = moves[random.randint(0, len(moves)-1)]
        board.apply_move(m)
        continue
    terminal, winner = board.is_terminal()
    return winner

# white == white_model
# black == black_model
def model1_vsmodel2(board, model_white, model_black, to_network_white, to_network_black):
    record = []
    while(not fst(board.is_terminal())):
        if(board.turn == Board.WHITE):
            q = model_white.predict(np.array([to_network_white()]))
            masked_output = [0 for x in range(0,1296)]
            for m in board.generate_moves():
                m_idx = board.get_network_output_index(m)
                masked_output[m_idx] = q[0][0][m_idx]
            best_idx = np.argmax(masked_output)
            sel_move = None
            for m in board.generate_moves():
                m_idx = board.get_network_output_index(m)
                if(best_idx == m_idx):
                    sel_move = m
            board.apply_move(sel_move)
            record.append(sel_move)
            if len(record) > 200:
                return 0
            continue
        else:
            q = model_black.predict(np.array([to_network_black()]))
            masked_output = [0 for x in range(0,1296)]
            for m in board.generate_moves():
                m_idx = board.get_network_output_index(m)
                masked_output[m_idx] = q[0][0][m_idx]
            best_idx = np.argmax(masked_output)
            sel_move = None
            for m in board.generate_moves():
                m_idx = board.get_network_output_index(m)
                if(best_idx == m_idx):
                    sel_move = m
            board.apply_move(sel_move)
            record.append(sel_move)
            if len(record) > 200:
                return 0
            continue

    terminal, winner = board.is_terminal()
    return winner

In [ ]:
white_wins = 0
black_wins = 0
draws = 0

for i in tqdm(range(0,100)):
    board = Board()
    board.set_starting_position()
    moves = board.generate_moves()
    m = moves[random.randint(0, len(moves)-1)]
    board.apply_move(m)
    winner = rand_vs_rand(board)
    if(winner == Board.WHITE):
        white_wins += 1
    if(winner == Board.BLACK):
        black_wins += 1
    if winner == 0:
        draws += 1

all = white_wins + black_wins
print("Rand vs Rand: "+str(white_wins/all) + "/"+str(black_wins/all)+ "/"+str(draws/all))

In [ ]:
white_wins = 0
black_wins = 0
draws = 0

for i in tqdm(range(0,100)):
    board = Board()
    board.set_starting_position()
    moves = board.generate_moves()
    m = moves[random.randint(0, len(moves)-1)]
    board.apply_move(m)
    winner = net_vs_rand(board)
    if(winner == Board.WHITE):
        white_wins += 1
    if(winner == Board.BLACK):
        black_wins += 1
    if(winner == 0):
        draws += 1

all = white_wins + black_wins + draws
print("Rand Network vs Random: "+str(white_wins/all) + "/"+str(black_wins/all) + "/"+str(draws/all))


In [ ]:
white_wins = 0
black_wins = 0
draws = 0

for i in tqdm(range(0,100)):
    board = Board()
    board.set_starting_position()
    moves = board.generate_moves()
    m = moves[random.randint(0, len(moves)-1)]
    board.apply_move(m)
    winner = rand_vs_net(board)
    if(winner == Board.WHITE):
        white_wins += 1
    if(winner == Board.BLACK):
        black_wins += 1
    if(winner == 0):
        draws += 1

all = white_wins + black_wins + draws
print("Random vs Random Network: "+str(white_wins/all) + "/"+str(black_wins/all) + "/"+str(draws/all))

In [ ]:
#model = keras.models.load_model('saved_models/random_model_tablut.keras')
#model = keras.models.load_model('saved_models/supervised_model_tablut.keras')
model = keras.models.load_model('saved_models/supervised_model_tablut_md.keras')

In [6]:
model_1 = keras.models.load_model('saved_models/supervised_model_tablut.keras')
model_2 = keras.models.load_model('saved_models/supervised_model_tablut.keras')


white_wins = 0
black_wins = 0
draws = 0

for i in tqdm(range(0,100)):
    board = Board()
    board.set_starting_position()
    moves = board.generate_moves()
    m = moves[random.randint(0, len(moves)-1)]
    board.apply_move(m)
    winner = model1_vsmodel2(board, model_1, model_2,
                             board.to_network_input, board.to_network_input)
    if(winner == Board.WHITE):
        white_wins += 1
    if(winner == Board.BLACK):
        black_wins += 1
    if(winner == 0):
        draws += 1

all = white_wins + black_wins + draws
print("Network vs Network: "+str(white_wins/all) + "/"+str(black_wins/all) + "/"+str(draws/all))


  0%|          | 0/100 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━

In [ ]:
model = keras.models.load_model('saved_models/supervised_model_tablut_md.keras')

board = Board()
board.set_starting_position()

In [ ]:
if(board.turn == Board.WHITE):
    moves = board.generate_moves()
    m = ((28, 27))
    board.apply_move(m)
else:
    q = model.predict(np.array([board.to_network_input_multidim()]))
    masked_output = [0 for x in range(0,1296)]
    for m in board.generate_moves():
        m_idx = board.get_network_output_index(m)
        masked_output[m_idx] = q[0][0][m_idx]
    best_idx = np.argmax(masked_output)
    sel_move = None
    for m in board.generate_moves():
        m_idx = board.get_network_output_index(m)
        if(best_idx == m_idx):
            sel_move = m
    board.apply_move(sel_move)
    print(q)

draw_board(board.board)